## Langchain and OpenAI API to Extract Text and Table from PDF

In [1]:
# # - Test OpenAI_API_Key
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of loops and calls so pure,\nLies a technique that offers much allure.\nRecursion, a wondrous coding art,\nA poetic dance, a song from the heart.\n\nAt its core, a function calls its own name,\nEchoing through the code like a mystical flame.\nBreaking down problems both big and small,\nInto simpler tasks, it heeds the call.\n\nLike a nesting doll, layers upon layers,\nEach one a mirror of its neighbors.\nWith a base case to stop the chain,\nRecursion brings harmony, no task in vain.\n\nAs the function unwinds, gracefully it flows,\nSolving puzzles, revealing the path it knows.\nA beauty in simplicity, elegance in form,\nRecursion weaves magic, a coding norm.\n\nSo embrace the recursion, let it guide your hand,\nIn the world of programming, a symphony so grand.\nFor in its loops and calls, a masterpiece unfurled,\nRecursion, the enchanting dance of the coding world.', role='assistant', function_call=None, tool_calls=None)


In [2]:
# # - Test OpenAI Image Process
# from openai import OpenAI

# client = OpenAI()

# response = client.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#     {
#       "role": "user",
#       "content": [
#         {"type": "text", "text": "What’s in this image?"},
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
#           },
#         },
#       ],
#     }
#   ],
#   max_tokens=300,
# )

# print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This image depicts a scenic view of a lush green field with tall grass, some bushes, and scattered trees in the distance. In the foreground, there is a wooden boardwalk pathway that extends into the distance, appearing to guide the viewer’s eye toward the horizon. The sky above is bright blue with scattered white clouds, contributing to a serene, picturesque outdoor scene.', role='assistant', function_call=None, tool_calls=None))


In [4]:
# - Convert PDF to Images
import fitz
from PIL import Image
import os

def create_output_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def pdf_to_images(pdf_path, output_folder):
    create_output_folder(output_folder)
    pdf_document = fitz.open(pdf_path)
    
    for page_num in range(pdf_document.page_count):
        # Get the page
        page = pdf_document.load_page(page_num)
        
        # Render the page to a pixmap
        pix = page.get_pixmap()
        
        # Convert the pixmap to an image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Save the image with a name based on the page number
        img.save(os.path.join(output_folder, f"page_{page_num + 1}.png"))

    # Close the PDF document
    pdf_document.close()

# Example usage
pdf_path = 'RoHS.pdf'
output_folder = 'pdf_to_images'

pdf_to_images(pdf_path, output_folder)


In [8]:
# # - Test One PDF Image
# import base64
# import requests
# import os

# # OpenAI API Key
# api_key = os.getenv("OPENAI_API_KEY")

# # Function to encode the image
# def encode_image(image_path):
#   with open(image_path, "rb") as image_file:
#     return base64.b64encode(image_file.read()).decode('utf-8')

# # Path to your image
# image_path = "./pdf_to_images/page_1.png"

# # Getting the base64 string
# base64_image = encode_image(image_path)

# headers = {
#   "Content-Type": "application/json",
#   "Authorization": f"Bearer {api_key}"
# }

# payload = {
#   "model": "gpt-4o",
#   "messages": [
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "Please abstract a list of amend rules following 'Amended by: ...'"
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{base64_image}"
#           }
#         }
#       ]
#     }
#   ],
#   "max_tokens": 300
# }

# response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# print(response.json())

{'id': 'chatcmpl-9mCNgpE3l2SPrBoHhJ896lDzHL82v', 'object': 'chat.completion', 'created': 1721274364, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the list of amended rules following 'Amended by: ...':\n\n1. **M1**: Commission Delegated Directive 2012/50/EU of 10 October 2012, published in Official Journal L 348, page 16 on 18.12.2012\n2. **M2**: Commission Delegated Directive 2012/51/EU of 10 October 2012, published in Official Journal L 348, page 18 on 18.12.2012\n3. **M3**: Commission Delegated Directive 2014/1/EU of 18 October 2013, published in Official Journal L 4, page 45 on 9.1.2014\n4. **M4**: Commission Delegated Directive 2014/2/EU of 18 October 2013, published in Official Journal L 4, page 47 on 9.1.2014\n5. **M5**: Commission Delegated Directive 2014/3/EU of 18 October 2013, published in Official Journal L 4, page 49 on 9.1.2014\n6. **M6**: Commission Delegated Directive 2014/4/EU of 18 October 2013, published i

In [39]:
# # - Test Multiple Local Images
# import base64
# import requests
# import os
# import json

# # OpenAI API Key
# api_key = os.getenv("OPENAI_API_KEY")

# # Function to encode the image
# def encode_image(image_path):
#   with open(image_path, "rb") as image_file:
#     return base64.b64encode(image_file.read()).decode('utf-8')

# # Paths to your local images
# image_path1 = './pdf_to_images/page_1.png'
# image_path2 = './pdf_to_images/page_2.png'
# image_path3 = './pdf_to_images/page_3.png'
# image_path4 = './pdf_to_images/page_4.png'

# # Encode images in base64
# encoded_image1 = encode_image(image_path1)
# encoded_image2 = encode_image(image_path2)
# encoded_image3 = encode_image(image_path3)
# encoded_image4 = encode_image(image_path4)

# headers = {
#   "Content-Type": "application/json",
#   "Authorization": f"Bearer {api_key}"
# }

# payload = {
#   "model": "gpt-4o",
#   "messages": [
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "Please abstract a list of amend records following 'Amended by: ...' by combining sequential images"
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{encoded_image1}"
#           }
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{encoded_image2}"
#           }
#         }
#       ]
#     }
#   ],
#   "max_tokens": 2000
# }

# response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# response_data = response.json()
# with open('response.json', 'w') as json_file:
#     json.dump(response_data, json_file, indent=2)

# print("Response saved to response.json")

# print(response.json())



Response saved to response.json
{'id': 'chatcmpl-9mCxt1YihlI3DTcfEgNbufwvalqKH', 'object': 'chat.completion', 'created': 1721276609, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Here is the abstracted list of "Amended by" records in the provided images:\n\n1. M1: Commission Delegated Directive 2012/50/EU of 10 October 2012: Official Journal L 348, pages 16, dated 18.12.2012.\n2. M2: Commission Delegated Directive 2012/51/EU of 10 October 2012: Official Journal L 348, pages 18, dated 18.12.2012.\n3. M3: Commission Delegated Directive 2014/1/EU of 18 October 2013: Official Journal L 4, pages 45, dated 9.1.2014.\n4. M4: Commission Delegated Directive 2014/2/EU of 18 October 2013: Official Journal L 4, pages 47, dated 9.1.2014.\n5. M5: Commission Delegated Directive 2014/3/EU of 18 October 2013: Official Journal L 4, pages 49, dated 9.1.2014.\n6. M6: Commission Delegated Directive 2014/4/EU of 18 October 2013: Official Journal L 4, pag

In [40]:
with open('response.json', 'r') as json_file:
    saved_response = json.load(json_file)

content = saved_response['choices'][0]['message']['content']
content.split('\n')

['Here is the abstracted list of "Amended by" records in the provided images:',
 '',
 '1. M1: Commission Delegated Directive 2012/50/EU of 10 October 2012: Official Journal L 348, pages 16, dated 18.12.2012.',
 '2. M2: Commission Delegated Directive 2012/51/EU of 10 October 2012: Official Journal L 348, pages 18, dated 18.12.2012.',
 '3. M3: Commission Delegated Directive 2014/1/EU of 18 October 2013: Official Journal L 4, pages 45, dated 9.1.2014.',
 '4. M4: Commission Delegated Directive 2014/2/EU of 18 October 2013: Official Journal L 4, pages 47, dated 9.1.2014.',
 '5. M5: Commission Delegated Directive 2014/3/EU of 18 October 2013: Official Journal L 4, pages 49, dated 9.1.2014.',
 '6. M6: Commission Delegated Directive 2014/4/EU of 18 October 2013: Official Journal L 4, pages 51, dated 9.1.2014.',
 '7. M7: Commission Delegated Directive 2014/5/EU of 18 October 2013: Official Journal L 4, pages 53, dated 9.1.2014.',
 '8. M8: Commission Delegated Directive 2014/6/EU of 18 October 2

In [41]:
# - Test Tabular Text
import base64
import requests
import os

# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "./pdf_to_images/page_45.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Please abstract tabular data in a dictionary data structure which I will save it in json later'"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())


{'id': 'chatcmpl-9mD76c2aOYmB09HfPaevQxdenjZ6T', 'object': 'chat.completion', 'created': 1721277180, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Here is the tabular data abstracted into a dictionary format suitable for saving as JSON:\n\n```python\ndata = {\n    "Directive 2002/95/EC": {\n        "Article 1": "Article 1",\n        "Article 2(1)": "Article 2(1), 2(2), Annex I",\n        "Article 2(2)": "Article 2(3)",\n        "Article 2(3)": "Article 2(4), introductory wording",\n        "None_1": "Article 2(4)",\n        "Article 3(a)": "Article 3(1)(2)",\n        "Article 3(b)": "None",\n        "None_2": "Article 3(6)-(28)",\n        "Article 4(1)": "Article 4(1), Annex II",\n        "None_3": "Article 4(3)-(4)",\n        "Article 4(2)": "Article 4(6)",\n        "Article 4(3)": "None",\n        "Article 5(1), introductory wording": "Article 5(1), introductory wording",\n        "Article 5(1)(a)": "Article 4(2)",\n        "Artic

**Response**:
{'id': 'chatcmpl-9mCNgpE3l2SPrBoHhJ896lDzHL82v', 'object': 'chat.completion', 'created': 1721274364, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the list of amended rules following 'Amended by: ...':\n\n1. **M1**: Commission Delegated Directive 2012/50/EU of 10 October 2012, published in Official Journal L 348, page 16 on 18.12.2012\n2. **M2**: Commission Delegated Directive 2012/51/EU of 10 October 2012, published in Official Journal L 348, page 18 on 18.12.2012\n3. **M3**: Commission Delegated Directive 2014/1/EU of 18 October 2013, published in Official Journal L 4, page 45 on 9.1.2014\n4. **M4**: Commission Delegated Directive 2014/2/EU of 18 October 2013, published in Official Journal L 4, page 47 on 9.1.2014\n5. **M5**: Commission Delegated Directive 2014/3/EU of 18 October 2013, published in Official Journal L 4, page 49 on 9.1.2014\n6. **M6**: Commission Delegated Directive 2014/4/EU of 18 October 2013, published in Official Journal L 4, page 51 on 9.1.2014\n7. **M7**: Commission Deleg"}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 787, 'completion_tokens': 300, 'total_tokens': 1087}, 'system_fingerprint': 'fp_5e997b69d8'}